In [1]:
import os
import pandas as pd
import numpy as np
import ollama
import requests

In [2]:
data_path = "data"
current_part = "pickle_part_9.pkl"
current_pickle_file = os.path.join(data_path, current_part)

In [ ]:
import os
import pandas as pd

# Đọc và gộp dữ liệu từ tất cả file .pkl - # Read and merge data from all .pkl files
df = pd.read_pickle(os.path.join(current_pickle_file))



In [ ]:
df.iloc[1, :]

In [ ]:
# Convert them to lyrics - Chuyển đổi chúng thành lời bài hát
def convert_to_lyrics(df):
    df['lyrics'] = df['text'].apply(lambda x: x.replace('\n', ' ').strip())
    return df[['id', 'lyrics']]

In [ ]:
import requests

OLLAMA_HOST = "http://localhost:11434"

def generate_response(model: str = "nomic-embed-text", prompt: str = "The sky is blue because of Rayleigh scattering"):
    url = f"{OLLAMA_HOST}/api/embeddings"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False  # Set to True if you want streaming
    }

    response = requests.post(url, json=payload)
    response.raise_for_status()  # Throw if error
    try:
        return response.json().get("embedding")
    except Exception as e:
        return response.text

In [ ]:
import logging
import time
# Configure logging to a file with timestamps, create in the current directory
logging_file_name = f"embedding_log_{time.strftime('%Y%m%d_%H%M%S')}.log"
logging.basicConfig(filename=logging_file_name, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')



In [ ]:
for i, lyric in enumerate(lyrics[0:100]):
    try:
        logging.info(f"Processing lyric index: {i}")
        embeddings = generate_response(prompt=lyric)
        print(f"Generated embeddings for lyric index {i}: {embeddings}")
        logging.info(f"Generated embeddings for lyric index {i}")
        # Chroma Database integration would go here
    except Exception as e:
        logging.error(f"Error processing lyric index {i}: {e}")
        logging.warning(f"Skipping lyric index {i} with lyrics: {lyrics}")

In [ ]:
import os
import pandas as pd
import requests
import chromadb
import logging
import time


chroma_db_path = r"D:\ĐỒ ÁN TỐT NGHIỆP\all_pickles\chroma_db"


chroma_client = chromadb.PersistentClient(path=chroma_db_path)
collection = chroma_client.get_or_create_collection(name="lyrics_embeddings")

# Logging
logging_file_name = f"embedding_log_{time.strftime('%Y%m%d_%H%M%S')}.log"
logging.basicConfig(filename=logging_file_name, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# URL OLLAMA
OLLAMA_HOST = "http://localhost:11434"

def generate_response(model: str = "nomic-embed-text", prompt: str = "The sky is blue because of Rayleigh scattering"):
    url = f"{OLLAMA_HOST}/api/embeddings"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }

    response = requests.post(url, json=payload)
    response.raise_for_status()
    try:
        return response.json().get("embedding")
    except:
        return response.text

# Đường dẫn thư mục chứa các file .pkl
data_folder = r"D:\ĐỒ ÁN TỐT NGHIỆP\all_pickles\data"

# Biến đếm tổng embeddings
embedding_count = 0

# Duyệt toàn bộ file .pkl trong thư mục
for file_name in os.listdir(data_folder):
    if file_name.endswith(".pkl"):
        file_path = os.path.join(data_folder, file_name)
        logging.info(f"Processing file: {file_name}")

        # Đọc DataFrame từ file .pkl
        df = pd.read_pickle(file_path)
 
        # Lấy danh sách lyrics
        lyrics = df['lyrics'].dropna().tolist()

        for i, lyric in enumerate(lyrics):
            try:
                logging.info(f"Processing lyric index {i} in file {file_name}")
                embedding = generate_response(prompt=lyric)

                # Lưu vào ChromaDB persistent
                collection.add(
                    ids=[f"{file_name}_{i}"],
                    documents=[lyric],
                    embeddings=[embedding]
                )

                embedding_count += 1
                logging.info(f"Generated and saved embedding for index {i} in file {file_name}")

            except Exception as e:
                logging.error(f"Error processing lyric index {i} in file {file_name}: {e}")
                logging.warning(f"Skipping lyric index {i} in file {file_name}")

print(f"✅ Migrated to new architecture and saved {embedding_count} embeddings vào ChromaDB!")
